# Knuckledragger: A Low Barrier Proof Assistant
## Philip Zucker
### Draper Laboratory

![](https://raw.githubusercontent.com/philzook58/knuckledragger/main/docs/logo.webp)


# What is it?

- A library
- An interactive theorem prover in Python
- Maximally shallow wrapping around Z3
- Rigorous chaining of SMT solves
- Piles of little logical gizmos (rewriters, unifiers, printers, etc)

# Demo

In [1]:
import kdrag as kd
import kdrag.smt as smt # smt is literally a reexporting of z3

# Knuckledragger support algebraic datatypes and induction
Nat = kd.Inductive("MyNat")
Zero = Nat.declare("Zero")
Succ = Nat.declare("Succ", ("pred", Nat))
Nat = Nat.create()

In [2]:
# We can define an addition function by cases
n,m = smt.Consts("n m", Nat)
add = smt.Function("add", Nat, Nat, Nat)
add = kd.define("add", [n,m], 
    kd.cond(
        (n.is_Zero, m),
        (n.is_Succ, Nat.Succ(add(n.pred, m)))
))

kd.notation.add.register(Nat, add)

In [3]:
add_zero_x = kd.prove(smt.ForAll([n], Nat.Zero + n == n), by=[add.defn])
add_succ_x = kd.prove(smt.ForAll([n,m], Nat.Succ(n) + m == Nat.Succ(n + m)), by=[add.defn])

In [4]:
l = kd.Lemma(smt.ForAll([n], n + Nat.Zero == n))
_n = l.fix()            
l.induct(_n)              
l.auto(by=[add.defn])
l.auto(by=[add.defn])
add_x_zero = l.qed()

# Inference Rules

- Hilbert style system
- Proof objects are basically trees of recorded z3 calls

```markdown
t1 True  t2 True ....    Not(Implies(And(t1,t2,...), t)) unsat    
---------------------------------------------------------------- prove
                           t True
```

In [17]:
x = smt.Int("x")
xpos = kd.axiom(x > 0)
# ---------------------------
kd.kernel.prove(x > -1, by=[xpos])

|- x > -1

In [29]:
pf = kd.prove(x == x)
type(pf)

kdrag.kernel.Proof

In [31]:
type(pf.thm)

z3.z3.BoolRef

In [34]:
kd.kernel.prove(x > -1, by=[xpos]).reason

[|- x > 0]

Z3 needs help with quantifiers

```markdown
  ∀x:T, P(x) Formula     y : T Fresh
---------------------------------- herb
    P(y) => ∀x, P(x) True    
```

In [16]:
vs, pf = kd.kernel.herb(smt.ForAll([x], x > 0))
pf

|- Implies(x!1195 > 0, ForAll(x, x > 0))

```markdown
∀x:T, P(x) True      t : T
--------------------------------- instan
         P(t) True
```

In [22]:
pf = kd.prove(smt.ForAll([x], x > x - 1))
kd.kernel.instan([smt.IntVal(42)], pf)

|- 42 > 42 - 1




```markdown
    t : T       P : T -> Bool    T inductive 
----------------------------------------- datatype induct
     (/\_{C_T} (∀y, P(y) => P(C(y)))) => P(t) True
```

As an example
```
      t : Nat      
-------------------------------------------------- Nat induct
  (P(Z) /\ (∀y, P(y) => P(Succ(y)))) => P(t) True
```



Axiom schema

In [27]:
import kdrag.theories.nat as nat
n = smt.Const("n", nat.Nat)
P = smt.Function("P", nat.Nat, smt.BoolSort())
kd.kernel.induct_inductive(n, P)

|- Implies(And(P(Z),
            ForAll(pred!1202,
                   Implies(P(pred!1202), P(S(pred!1202))))),
        P(n))

# System Comparison

- Isabelle
- Lean/Coq
- ACL2
- Why3 / Boogie

# Backwards Tactics

- Mutable `Lemma` object
- Record `Proof` of steps
- list of goals
- `l.qed` assembles steps via `prove call

# Subsystems

- Reflection
- Quickcheck using hypothesis
- Library Search
- Typeclasses and Generics - https://www.philipzucker.com/knuckle_typeclass/
- Solvers. KB, Prolog, EProver

# Theories


In [5]:
import kdrag.theories.real as real
import kdrag.theories.bool as bool_
import kdrag.theories.bitvec as bitvec
import kdrag.theories.seq as seq
import kdrag.theories.nat as nat
import kdrag.theories.int as int_
import kdrag.theories.list as list_
import kdrag.theories.option as option
import kdrag.theories.set as set_
import kdrag.theories.real.complex as complex
import kdrag.theories.algebra.group as group
import kdrag.theories.algebra.lattice
import kdrag.theories.algebra.ordering
import kdrag.theories.fixed
import kdrag.theories.float
import kdrag.theories.real.arb
import kdrag.theories.real.sympy
import kdrag.theories.real.vec
import kdrag.theories.logic.intuitionistic
import kdrag.theories.logic.temporal

Admitting lemma ForAll([t, s], mul(expi(t), expi(s)) == expi(t + s))


       norm2(u) ==
       0 + x0(u)*x0(u) + x1(u)*x1(u) + x2(u)*x2(u)) to ForAll(u,
       norm2(u) == x0(u)*x0(u) + x1(u)*x1(u) + x2(u)*x2(u))


# Quickcheck


In [6]:
import kdrag.hypothesis as hyp
import kdrag.theories.nat as nat
n = smt.Const("n", nat.Nat)
hyp.quickcheck(smt.ForAll([n], n + nat.Z == n))

In [7]:
hyp.quickcheck(smt.ForAll([n], n + nat.Z == n + nat.one))

AssertionError: ('Found a counterexample', [])

# Shallow Embedding of Logics

In [ ]:
import kdrag.theories.logic.intuitionistic as intuitionistic
import kdrag.theories.logic.temporal as temporal


# Search

In [8]:
x = smt.Real("x")
kd.search(real.cos(x)**2)

{('kdrag.theories.real.pythag_1',
  |- ForAll(x, cos(x)**2 == 1 - sin(x)**2)): [x],
 ('kdrag.theories.real.cos_neg', |- ForAll(x, cos(-x) == cos(x))): [x]}

# Rewriting


In [10]:
x,y,z = smt.Reals("x y z")
unit = kd.prove(smt.ForAll([x], x + 0 == x))
mul_zero = kd.prove(smt.ForAll([x], x * 0 == 0))
kdrag.rewrite.rewrite(0 + x + y*0 + z + 0 + x*0, [unit, mul_zero])

0 + x + z

In [11]:
smt.simplify(0 + x + y*0 + z + 0 + x*0)

x + z

In [ ]:
kd.utils.unify([x,y,z], x + (x + z), y + (x + 0))

{z: 0, x: y}

Locally nameless combinator

In [ ]:
kd.utils.open_binder(smt.ForAll([x,y], x + y == y + x))

([X!10962, Y!10963], X!10962 + Y!10963 == Y!10963 + X!10962)

In [ ]:
kd.utils.alpha_eq(smt.ForAll([x], x + 0 == x), smt.ForAll([y], y + 0 == y))

True

# Knuth Bendix Completion

In [ ]:
import kdrag.solvers.kb as kb
# https://www.philipzucker.com/knuth_bendix_knuck/
T = smt.DeclareSort("AbstractGroup")
x,y,z = smt.Consts("x y z", T)
e = smt.Const("a_e", T)
inv = smt.Function("c_inv", T, T)
mul = smt.Function("b_mul", T, T, T)
kd.notation.mul.register(T, mul)
kd.notation.invert.register(T, inv)
E = [
    smt.ForAll([x], e * x == x),
    smt.ForAll([x], inv(x) * x == e),
    smt.ForAll([x,y,z], (x * y) * z == x * (y * z)),
]
kb.huet(E, order=kd.rewrite.lpo)

[RewriteRule(vs=[Z!10564, X!10565], lhs=c_inv(b_mul(X!10565, Z!10564)), rhs=b_mul(c_inv(Z!10564), c_inv(X!10565)), pf=None),
 RewriteRule(vs=[Z!7894, X!7895], lhs=b_mul(X!7895, b_mul(c_inv(X!7895), Z!7894)), rhs=Z!7894, pf=None),
 RewriteRule(vs=[X!7885], lhs=c_inv(c_inv(X!7885)), rhs=X!7885, pf=None),
 RewriteRule(vs=[], lhs=c_inv(a_e), rhs=a_e, pf=None),
 RewriteRule(vs=[X!7880], lhs=b_mul(X!7880, c_inv(X!7880)), rhs=a_e, pf=None),
 RewriteRule(vs=[X!7611], lhs=b_mul(X!7611, a_e), rhs=X!7611, pf=None),
 RewriteRule(vs=[Z!7529, X!7530], lhs=b_mul(c_inv(X!7530), b_mul(X!7530, Z!7529)), rhs=Z!7529, pf=None),
 RewriteRule(vs=[X!7493], lhs=b_mul(a_e, X!7493), rhs=X!7493, pf=None),
 RewriteRule(vs=[X!7490], lhs=b_mul(c_inv(X!7490), X!7490), rhs=a_e, pf=None),
 RewriteRule(vs=[X!7485, Y!7486, Z!7487], lhs=b_mul(b_mul(X!7485, Y!7486), Z!7487), rhs=b_mul(X!7485, b_mul(Y!7486, Z!7487)), pf=None)]

# External Solvers

In [ ]:
s = kd.solvers.EProverTHFSolver()
for eq in E:
    s.add(eq)
s.add(smt.ForAll([x], x * e != x))
s.check()
print(s.res.stdout.decode())


# Preprocessing class: HSSSSMSSSSSNFFN.
# Scheduled 4 strats onto 8 cores with 300 seconds (2400 total)
# Starting new_ho_10 with 1500s (5) cores
# Starting ho_unfolding_6 with 300s (1) cores
# Starting sh4l with 300s (1) cores
# Starting ehoh_best_nonlift_rwall with 300s (1) cores
# ho_unfolding_6 with pid 1176906 completed with status 0
# Result found by ho_unfolding_6
# Preprocessing class: HSSSSMSSSSSNFFN.
# Scheduled 4 strats onto 8 cores with 300 seconds (2400 total)
# Starting new_ho_10 with 1500s (5) cores
# Starting ho_unfolding_6 with 300s (1) cores
# No SInE strategy applied
# Search class: HUUPS-FFSF21-SFFFFFNN
# Scheduled 6 strats onto 1 cores with 300 seconds (300 total)
# Starting new_ho_10 with 163s (1) cores
# new_ho_10 with pid 1176910 completed with status 0
# Result found by new_ho_10
# Preprocessing class: HSSSSMSSSSSNFFN.
# Scheduled 4 strats onto 8 cores with 300 seconds (2400 total)
# Starting new_ho_10 with 1500s (5) cores
# Starting ho_unfolding_6 with 300s (1

In [ ]:
s = kd.solvers.VampireSolver()
for eq in E:
    s.add(eq)
s.add(smt.ForAll([x], x * e != x))
s.check()
print(s.res.stdout.decode())

(set-logic ALL)

(declare-sort AbstractGroup 0)

;;declarations

(declare-fun b_mul_2d4 (AbstractGroup AbstractGroup) AbstractGroup)

(declare-fun a_e_22a () AbstractGroup)

(declare-fun c_inv_34e (AbstractGroup) AbstractGroup)

;;axioms

(assert (forall ((X!10954_398 AbstractGroup)) (= (b_mul_2d4 a_e_22a X!10954_398) X!10954_398)))

(assert (forall ((X!10955_33c AbstractGroup)) (= (b_mul_2d4 (c_inv_34e X!10955_33c) X!10955_33c) a_e_22a)))

(assert (forall ((X!10956_398 AbstractGroup) (Y!10957_33c AbstractGroup) (Z!10958_322 AbstractGroup)) (= (b_mul_2d4 (b_mul_2d4 X!10956_398 Y!10957_33c) Z!10958_322) (b_mul_2d4 X!10956_398 (b_mul_2d4 Y!10957_33c Z!10958_322)))))

(assert (forall ((X!10959_3a1 AbstractGroup)) (distinct (b_mul_2d4 X!10959_3a1 a_e_22a) X!10959_3a1)))

(check-sat)

b'WARNING Broken Constraint: if inline_let(on) has been set then newcnf(off) is equal to on\nWARNING Broken Constraint: if inline_let(on) has been set then newcnf(off) is equal to on\nWARNING Broken Constraint

# Reflection



In [ ]:
import kdrag.reflect as reflect

@reflect.reflect
def fact(x : int) -> int:
    if x <= 0:
        return 1
    else:
        return x*fact(x-1)


Z3Exception: sort mismatch

# Applications 
Examples? I want to talk about stuff that isn't just applications Sheffer stroke

- Software foundations
- Ghidra pypcode
- Verilog - smt importing
- Sympy
- Arb

# Bits and Bobbles

```markdown
          P(t) True
--------------------------------- ∃I
        ∃x P(x) True
```


The world is a big place. Part of how to work in it is to Jeet Kune Do on what already exists or is popular.

Software and Hardware verification boils down to a bucnh of SMT queries with handwaving in between
I found myself writing a sequence of smt calls and then knowing that I'm reusing previous proved stuff


What is knuckeldragger 1 slide
1. The shallowest possible layer over z3 to make it compsoablte a proof system
The fumbest thing that could work


z3 prove is
def prove(e : smt.BoolRef) -> bool:
    s = smt.Solver()
    s.add(smt.Not(e))
    smt.check()
    if s.check() == smt.unsat:
        return True
    else:
        return True
boolean blindness

a parse tree is a trace of a maethod that says is there a valid parse.

Certificates of the SMT processes is hard.
The meta chainsing of SMT calls is not that hard.

Exhuastively failing to find counterexamples.


LCF style theorem proving
3/2 types formulas expressions and proof
Anything goes on terms
pf -> tm
details of proof don't matter, but basically they are recording the call tree that produced them.
inference rules ~ functions
COmplete erasure pf ~ tm is fine


The big inference rule is

t1 proved  t2 proved ...     Not(t1 /\ t2/\ t3 => t)  z3unsat
---------------------------------------------------------
                      t proved
mega modus ponens


But not actually

Definitions
Quantifier Instantiation
Induction

Tactics

Why python?
What are proofs

# Subsystems

- Reflection
- Quickcheck
- Typeclasses
- Generics

# Applications

